##### Deep Learning Pipeline for Motor Imagery Classification

*Data Preparation*

EEG epochs are extracted as input data X, shaped as (trials, channels, samples), with labels y marking the task type (left hand, right hand, feet, tongue). Each trial is normalized using z-scoring to stabilize variance across trials, and an additional dimension is added so the data matches CNN input requirements (trials, channels, samples, 1). Preprocessing includes band-pass filtering (8–30 Hz), removal of EOG channels, and setting electrodes to the standard 10–20 montage.
<br>
<br>



*Model (EEGNet)*

The model is a convolutional neural network (CNN) designed specifically for EEG data. It applies temporal convolutions to capture frequency-specific patterns, depthwise spatial convolutions to model motor-related activity across electrodes, and separable convolutions for efficient feature extraction. Regularization techniques such as batch normalization, dropout, and average pooling are included to stabilize training and reduce overfitting. The final dense layer with softmax activation outputs probabilities for the four motor imagery classes.
<br>
<br>



*Training and Validation*

Instead of pooling all subjects together, models are trained and validated separately per subject. Each subject’s dataset is split into 80% training and 20% testing, with stratification to preserve class balance. Training runs for up to 100 epochs using the Adam optimizer, and evaluation includes accuracy, classification reports, and confusion matrices.
<br>
<br>



*Results and Interpretation*

When training on pooled data from all subjects, the model achieved accuracies close to chance level (~25–35%), with mean cross-validation performance around 30%. Confusion matrices showed class collapse in several folds, where the model predicted only one or two classes. This reflects the difficulty of cross-subject generalization, since motor imagery EEG patterns are highly individual and vary in amplitude, frequency distribution, and electrode topography.

By shifting to subject-specific training, performance improved markedly for several subjects. Stronger subjects (e.g., A01T, A08T, A09T) reached accuracies above 65–75%, with balanced confusion matrices showing clear class separability. Weaker subjects (e.g., A02T, A05T, A06T) remained closer to chance, but overall the subject-specific approach provided higher and more stable accuracies compared to the pooled model.

This confirms that motor imagery EEG classification benefits from individualized training, as the model adapts to each subject’s unique neural signatures rather than trying to generalize across highly variable brain patterns. Based on these findings, the next step in my project is to integrate Common Spatial Pattern (CSP) features with EEGNet, aiming to enhance discriminability by combining handcrafted spatial filters with deep feature learning.

In [ ]:
import os
import json 
import mne
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, DepthwiseConv2D, SeparableConv2D
from tensorflow.keras.layers import BatchNormalization, Activation, AveragePooling2D, Dropout, Flatten, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

data_path = "BCICIV_2a_gdf/"
subjects = [f"A0{i}T.gdf" for i in range(1, 10)]  # A01T–A09T

# Define save directory
save_dir = "saved_models"
os.makedirs(save_dir, exist_ok=True)

# Channel mapping
mapping = {
    'EEG-Fz': 'Fz', 'EEG-0': 'FC3', 'EEG-1': 'FC1', 'EEG-2': 'FCz',
    'EEG-3': 'FC2', 'EEG-4': 'FC4', 'EEG-5': 'C5', 'EEG-C3': 'C3',
    'EEG-6': 'C1', 'EEG-Cz': 'Cz', 'EEG-7': 'C2', 'EEG-C4': 'C4',
    'EEG-8': 'C6', 'EEG-9': 'CP3', 'EEG-10': 'CP1', 'EEG-11': 'CPz',
    'EEG-12': 'CP2', 'EEG-13': 'CP4', 'EEG-14': 'P1', 'EEG-Pz': 'Pz',
    'EEG-15': 'P2', 'EEG-16': 'POz'
}

def EEGNet(nb_classes=4, Chans=22, Samples=751, dropoutRate=0.5):
    input1 = Input(shape=(Chans, Samples, 1))

    # Block 1
    block1 = Conv2D(8, (1, 64), padding='same', use_bias=False)(input1)
    block1 = BatchNormalization()(block1)
    block1 = DepthwiseConv2D((Chans, 1), use_bias=False, depth_multiplier=2)(block1)
    block1 = BatchNormalization()(block1)
    block1 = Activation('elu')(block1)
    block1 = AveragePooling2D((1, 4))(block1)
    block1 = Dropout(dropoutRate)(block1)

    # Block 2
    block2 = SeparableConv2D(16, (1, 16), padding='same', use_bias=False)(block1)
    block2 = BatchNormalization()(block2)
    block2 = Activation('elu')(block2)
    block2 = AveragePooling2D((1, 8))(block2)
    block2 = Dropout(dropoutRate)(block2)

    # Classification
    flatten = Flatten()(block2)
    dense = Dense(nb_classes, activation='softmax')(flatten)

    return Model(inputs=input1, outputs=dense)


In [ ]:
# Subject-specific Training
for subj in subjects:
    subj_name = subj.replace(".gdf", "")
    print(f"\n=== Training on {subj_name} ===")
    raw = mne.io.read_raw_gdf(os.path.join(data_path, subj), preload=True)

    # Preprocessing
    raw.filter(8., 30., method="fir")
    raw.rename_channels(mapping)
    for ch in ['EOG-left', 'EOG-central', 'EOG-right']:
        if ch in raw.ch_names:
            raw.drop_channels([ch])
    raw.set_montage('standard_1020')

    # Extract events
    events, event_id_raw = mne.events_from_annotations(raw)
    print("Event IDs found:", event_id_raw)

    # Full mapping (BCI IV-2a codes)
    all_event_id = {'left_hand': 7, 'right_hand': 8, 'feet': 9, 'tongue': 10}

    # Only keep events available in this subject (preserve order of all_event_id)
    available_ids = set(np.unique(events[:, -1]))
    event_id = {k: v for k, v in all_event_id.items() if v in available_ids}
    print("Using event_id for this subject:", event_id)

    if len(event_id) < 2:
        print("⚠️ Skipping subject because too few classes available.")
        continue

    # Epoching
    picks = mne.pick_types(raw.info, eeg=True, eog=False)
    epochs = mne.Epochs(raw, events, event_id=event_id,
                        tmin=0.5, tmax=3.5,
                        picks=picks, baseline=None, preload=True)

    X = epochs.get_data()  # (trials, chans, samples)

    # Remap labels {7,8,9,10} -> {0,1,2,3} dynamically based on event_id order
    label_map = {v: i for i, v in enumerate(event_id.values())}
    y = np.array([label_map[e[-1]] for e in epochs.events], dtype=int)

    # simple per-trial normalization
    X = (X - X.mean(axis=2, keepdims=True)) / (X.std(axis=2, keepdims=True) + 1e-6)

    # Add channel dimension
    X = X[..., np.newaxis]  # (trials, chans, samples, 1)

    print(f"  -> X shape: {X.shape}, y shape: {y.shape}, classes: {np.unique(y)}")

    # Split into train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    # Build & compile model
    model = EEGNet(nb_classes=len(np.unique(y)), Chans=X.shape[1], Samples=X.shape[2])
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(0.001),
                  metrics=['accuracy'])

    # Train
    model.fit(
        X_train, y_train,
        batch_size=32,
        epochs=100,
        validation_data=(X_test, y_test),
        verbose=1
    )

    # Evaluate
    y_pred = np.argmax(model.predict(X_test), axis=1)
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred, digits=4))
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

    # Save model & metadata for this subject
    model_path = os.path.join(save_dir, f"EEGNet_{subj_name}.h5")
    model.save(model_path)

    meta = {
        "subject": subj_name,
        "event_id": event_id,                # e.g., {"left_hand":7, ...}
        "label_map": {str(k): int(v) for k, v in label_map.items()},  # {"7":0, "8":1, ...}
        "classes": list(event_id.keys()),    # ["left_hand","right_hand",...]
        "ch_names": epochs.ch_names,
        "input_shape": [int(X.shape[1]), int(X.shape[2]), 1]
    }
    with open(os.path.join(save_dir, f"EEGNet_{subj_name}_meta.json"), "w") as f:
        json.dump(meta, f, indent=2)

    print(f"✅ Saved model to {model_path}")
    print(f"✅ Saved metadata to {os.path.join(save_dir, f'EEGNet_{subj_name}_meta.json')}")


=== Training on A01T ===
Extracting EDF parameters from c:\Users\hamih\OneDrive\Desktop\personal projects\BCI\BCICIV_2a_gdf\A01T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...


c:\Users\hamih\anaconda3\Lib\contextlib.py:144: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


Reading 0 ... 672527  =      0.000 ...  2690.108 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 413 samples (1.652 s)

Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']
Event IDs found: {'1023': 1, '1072': 2, '276': 3, '277': 4, '32766': 5, '768': 6, '769': 7, '770': 8, '771': 9, '772': 10}
Using event_id for this subject: {'left_hand': 7, 'right_hand': 8, 'feet': 9, 'tongue': 10}
Not setting metadata
288 matching events found
No baseline cor


Classification Report:
              precision    recall  f1-score   support

           0     0.5455    0.4000    0.4615        15
           1     0.5833    1.0000    0.7368        14
           2     0.5000    0.4000    0.4444        15
           3     0.6364    0.5000    0.5600        14

    accuracy                         0.5690        58
   macro avg     0.5663    0.5750    0.5507        58
weighted avg     0.5648    0.5690    0.5473        58

Confusion Matrix:
[[ 6  7  1  1]
 [ 0 14  0  0]
 [ 4  2  6  3]
 [ 1  1  5  7]]
✅ Saved model to saved_models\EEGNet_A01T.h5
✅ Saved metadata to saved_models\EEGNet_A01T_meta.json

=== Training on A02T ===
Extracting EDF parameters from c:\Users\hamih\OneDrive\Desktop\personal projects\BCI\BCICIV_2a_gdf\A02T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, E

c:\Users\hamih\anaconda3\Lib\contextlib.py:144: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


...

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 413 samples (1.652 s)

Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']
Event IDs found: {'1023': 1, '1072': 2, '276': 3, '277': 4, '32766': 5, '768': 6, '769': 7, '770': 8, '771': 9, '772': 10}
Using event_id for this subject: {'left_hand': 7, 'right_hand': 8, 'feet': 9, 'tongue': 10}
Not setting metadata
288 matching events found
No baseline correction applied
0 projection items activated
Using data f

              precision    recall  f1-score   support

           0     0.7857    0.7333    0.7586        15
           1     0.7647    0.9286    0.8387        14
           2     0.7500    0.2000    0.3158        15
           3     0.6087    1.0000    0.7568        14

    accuracy                         0.7069        58
   macro avg     0.7273    0.7155    0.6675        58
weighted avg     0.7287    0.7069    0.6630        58

Confusion Matrix:
[[11  1  1  2]
 [ 0 13  0  1]
 [ 3  3  3  6]
 [ 0  0  0 14]]
✅ Saved model to saved_models\EEGNet_A08T.h5
✅ Saved metadata to saved_models\EEGNet_A08T_meta.json

=== Training on A09T ===
Extracting EDF parameters from c:\Users\hamih\OneDrive\Desktop\personal projects\BCI\BCICIV_2a_gdf\A09T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-P

c:\Users\hamih\anaconda3\Lib\contextlib.py:144: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 413 samples (1.652 s)

Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']
Event IDs found: {'1023': 1, '1072': 2, '276': 3, '277': 4, '32766': 5, '768': 6, '769': 7, '770': 8, '771': 9, '772': 10}
Using event_id for this subject: {'left_hand': 7, 'right_hand': 8, 'feet': 9, 'tongue': 10}
Not setting metadata
288 matching events found
No baseline correction applied
0 projection items activated
Using data f


Classification Report:
              precision    recall  f1-score   support

           0     0.7895    1.0000    0.8824        15
           1     0.6667    0.4286    0.5217        14
           2     0.6923    0.6000    0.6429        15
           3     0.5882    0.7143    0.6452        14

    accuracy                         0.6897        58
   macro avg     0.6842    0.6857    0.6730        58
weighted avg     0.6861    0.6897    0.6761        58

Confusion Matrix:
[[15  0  0  0]
 [ 2  6  3  3]
 [ 1  1  9  4]
 [ 1  2  1 10]]
✅ Saved model to saved_models\EEGNet_A09T.h5
✅ Saved metadata to saved_models\EEGNet_A09T_meta.json
